# Preparation of Custom Data for Upload to QA4SM

This notebook offers a brief introduction into how custom data can be prepared to be suitable for an upload to the [QA4SM validation service](https://qa4sm.eu/ui/my-datasets).
A detailed description of the upload option can be found [here](https://qa4sm.eu/ui/user-data-guidelines).

Data can be uploaded in two different file formats: 
1. Comma-separated values (*CSV*) files
2.  *NetCDF*  Image Stacks
   
This notebook offers a short walk-through for the data preparation for both options and can be used as a template for your own data preparation.

# Import of required packages and defining a data path and reader

First, all required packages are imported.

* `pynetcf` is a library for mapping to netCDF files on disk, written according to the [Climate and Forecast (CF) Metadata Conventions](http://cfconventions.org/). This library can be obtained from [github](https://github.com/TUW-GEO/pynetcf) or installed via `pip install pynetcf`.
* `smecv-grid` is a library containing the grid definition of the 0.25° Discrete Global Grid (DGG) used for the creation of the CCI soil moisture products and the Copernicus Climate Change Service products. This library can be obtained from [github](https://github.com/TUW-GEO/smecv-grid) or installed via `pip install smecv_grid`.

In [1]:
import os
from pathlib import Path
import pandas as pd
from  pynetcf.time_series import GriddedNcOrthoMultiTs
from smecv_grid.grid import SMECV_Grid_v052
from tqdm.notebook import tqdm
import zipfile

The satellite data used here is a combination of SMAP and SMOS L3 soil moisture, derived using the Land Parameter Retrieval Model. SMOS has been scaled to SMAP using a CDF matching method. Gaps in the data set have been filled using the Discrete Cosine Transform algorithm. The data is stored in a time series format. 

Lastly, the name of the folder to contain the results is defined here, too.

In [2]:
data = Path('custom_data', '020_gapfilled_timeseries')
out_folder ='time_series'

Then, the `reader` needed to extract a time series for a given pair of coordinates needs to be instantiated. 


In this case, the grid object `SMECV_Grid_v052`, referring to a global SMECV Grid and using WGS84 coordinates, is passed to the reader. Further, the keyword argument `read_bulk=True` is also passed to the reader, which allows to read the data in bulk. More information on the `GriddedNcOrthoMultiTs` reader can be found [here](https://github.com/TUW-GEO/pynetcf/blob/542a90dfcbb89e28a56e37f72dc08163375a742d/src/pynetcf/time_series.py#L1708).

In [3]:
reader = GriddedNcOrthoMultiTs(data, grid=SMECV_Grid_v052(), ioclass_kws={'read_bulk': True})

To validate the satellite data, Fiducial Reference Measurements (FRMs) are used. These are in situ soil moisture measurements. The FRMs were obtained and selected based on the following criteria:

* Soil moisture data has been downloaded in January 2023 from the [International Soil Moisture Network](https://ismn.geo.tuwien.ac.at/)
* The FRM classification is based on Triple Collocation Analysis between the in-situ data, ERA5-Land reanalysis and ESA CCI SM Passive data
* FRMs are selected based on minimizing random errors (via Signal to Noise Ratio) found in the in situ data that contain at least 100 days of soil moisture measurements

See [here](https://www.geo.tuwien.ac.at/media/filer_public/56/03/5603cc6a-46f8-44cd-a964-84f9d59301d9/frm4sm__dt2-1_fpp_sm_v04.pdf) for more details. **Link broken**

Only "**very representative**" sensors are used in this example. The following list contains the coordinates (Latitude and Longitude) of these FRMs. 

In [4]:
frms = pd.read_csv('./custom_data/frms.csv',
                   usecols=['idx', 'longitude', 'latitude'],
                   dtype={'idx': int, 'longitude': float, 'latitude': float},
                   engine='c',
                   index_col='idx')
frms

,longitude,latitude
idx,,
214,2.66040,13.53250
215,2.66040,13.53250
218,1.70994,9.79506
221,1.70994,9.79506
224,1.70994,9.78986
...,...,...
21279,96.34650,17.26300
21283,96.34650,17.26300
21307,96.45190,17.31440


# Calculation of GPIs corresponding to the location information for all FRMs

GPIs (Grid Point Indices) are found and added to the dataframe `frms` as a new column. As mutliple FRM locations can relate to the same exact GPI, all duplicate GPIs are removed from the dataframe.


In [5]:
tqdm.pandas(desc = 'Writing GPI column to DataFrame')  


def get_unique_gpis(df: pd.DataFrame) -> pd.DataFrame:

    def get_gpi(row: pd.Series) -> int:
        gpi, _ = reader.grid.find_nearest_gpi(row['longitude'], row['latitude'])
        return gpi

    df['gpi'] = df.progress_apply(get_gpi, axis=1)
    # df['gpi'] = df.apply(get_gpi, axis=1)             # if a progress bar is not desired
    df = df.drop_duplicates(subset = ['gpi'])    

    return df


unique_gpis = get_unique_gpis(frms)
unique_gpis

Writing GPI column to DataFrame:   0%|          | 0/604 [00:00<?, ?it/s]

,longitude,latitude,gpi
idx,,,
214,2.66040,13.53250,596890
218,1.70994,9.79506,575286
231,1.60530,9.74530,573846
288,-98.09690,37.21340,731847
291,-97.26600,37.13300,731850
...,...,...,...
21198,-101.44500,42.06800,760634
21236,-119.82080,37.75920,736080
21257,-1.27610,39.54930,746634


# Preparing the *ZIP* Archive containing the *CSV* files

Some pre-processing is required to prepare the data for upload to the QA4SM validation service in *CSV* file format. There are two distinct advantages to this:
1. This approach is especially useful for datasets with non-orthogonal time steps (i.e. different time stamps at different locations).
2. To reduce the amount of uploaded data (as e.g. only time series at in situ locations can be extracted beforehand) and thus use the storge space available for the QA4SM user more efficiently.
   

Certain criteria need to be fulfilled for such a *CSV* file, which are listed in the [user data upload guide](https://qa4sm.eu/ui/user-data-guidelines).

All *CSV* files have to be in the same directory. 

First, a small function is defined, taking a row of a DataFrame as input. Based on the given pair of coordinates and unique GPI contained in this row, the corresponding time series is parsed. Each time series is then written to a *CSV* file, which is saved in the pre-defined output directory.

In [6]:
def write_time_series_csv(row: pd.Series) -> None:
    time_series = reader.read(row['longitude'], row['latitude'])
    time_series.index.name = 'Time'

    fname = f"SMOSSMAP_gpi={row['gpi']}_lat={row['latitude']:.3f}_lon={row['longitude']:.3f}.csv"

    if not os.path.isdir(out_folder):
        os.mkdir(out_folder)

    time_series[['sm_gapfilled']].to_csv(os.path.join(out_folder, fname))

    return os.path.join(out_folder, fname)

This function is subsequently applied to all FRM locations.

In [7]:
tqdm.pandas(desc = f"Writing time series' CSV files to directory '{out_folder}'")
unique_gpis.progress_apply(write_time_series_csv, axis=1)
# unique_gpis.apply(write_time_series_csv, axis=1)         # if a progress bar is not desired

Writing time series' CSV files to directory 'time_series':   0%|          | 0/290 [00:00<?, ?it/s]

idx
214      time_series/SMOSSMAP_gpi=596890.0_lat=13.533_l...
218      time_series/SMOSSMAP_gpi=575286.0_lat=9.795_lo...
231      time_series/SMOSSMAP_gpi=573846.0_lat=9.745_lo...
288      time_series/SMOSSMAP_gpi=731847.0_lat=37.213_l...
291      time_series/SMOSSMAP_gpi=731850.0_lat=37.133_l...
                               ...                        
21198    time_series/SMOSSMAP_gpi=760634.0_lat=42.068_l...
21236    time_series/SMOSSMAP_gpi=736080.0_lat=37.759_l...
21257    time_series/SMOSSMAP_gpi=746634.0_lat=39.549_l...
21277    time_series/SMOSSMAP_gpi=618865.0_lat=17.263_l...
21329    time_series/SMOSSMAP_gpi=618866.0_lat=17.292_l...
Length: 290, dtype: object

To test whether these *CSV* files comply with the required format, following command can be used (here, only the first file is checked):

In [8]:
fname = os.listdir(out_folder)[0]
pd.read_csv(os.path.join(out_folder, fname), index_col=0, parse_dates=True)

,sm_gapfilled
Time,
2010-01-01,0.224519
2010-01-02,0.224840
2010-01-03,0.225556
2010-01-04,0.226809
2010-01-05,0.228789
...,...
2021-12-27,0.125000
2021-12-28,0.121507
2021-12-29,0.127000


Finally, all *CSV* files are compressed into a single *ZIP* archive. This is done by using a small helper function `zipdir` and the `zipfile` library

---
*NOTE*

Alternatively, the following bash command can be used as well: `!zip -j my_data.zip  time_series/*.csv`.

---

In [9]:
def zipdir(path, ziph):
    for file in os.listdir(path):
        ziph.write(os.path.join(os.path.abspath(path), file), 
                    os.path.relpath(os.path.join(os.path.abspath(path), file), 
                                    os.path.join(path, '..')))

with zipfile.ZipFile('Time_Series_CSV.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(out_folder, zipf)

# Alternative: *NetCDF*  Image Stacks

QA4SM also accepts *NetCDF*  image stacks as input. This is especially handy, as most level 3 and level 4 soil moisture products are distributed in this format. 

Yet, certain requirements regarding the *NetCDF*  file structure have to be met, for QA4SM to sucesfully make use of this data. These can be found in the [user data upload guide](https://qa4sm.eu/ui/user-data-guidelines).


Below, a short example on how to read a *NetCDF* image stack, using the [xarray](https://xarray.dev/) library, is given. For *NetCDF* stacks to be compatible with QA4SM, they should be of similar format as the example given below.

In [10]:
import xarray as xr

netcdffile = os.path.join('custom_data', 'europe_cci_gapfilled_2018_v3.nc')
ds = xr.open_dataset(netcdffile)
ds

<xarray.Dataset>
Dimensions:       (time: 365, lat: 150, lon: 238)
Coordinates:
  * time          (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-12-31
  * lon           (lon) float32 -11.12 -10.88 -10.62 ... 47.62 47.88 48.12
  * lat           (lat) float32 71.38 71.12 70.88 70.62 ... 34.62 34.38 34.12
Data variables:
    sm_gapfilled  (time, lat, lon) float32 ...
Attributes:
    id:                   ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED_GAPFILLED-201...
    time_coverage_start:  20180518T000000Z
    time_coverage_end:    20180518T235959Z
    date_created:         File created: 2023-05-30 04:42:32.834870

In this example, 150 values for the latitude `lat` and 238 for the longitude `lon` are contained in the *NetCDF* stack. The third dimesnion is the gapfilled soil moisture data `sm_gapfilled`, as function of `time`, `lat` and `lon`.